# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [8]:
import pandas as pd 

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [9]:
try: 
    df = pd.read_csv('/datasets/users_behavior.csv')
except: 
    df = pd.read_csv(r'C:\Users\QWE\Downloads\users_behavior.csv')

print(df.head(5))

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [10]:
df.info() #Общая информация
# В задании указано что данные обработаны и предобработка данных не понадобится 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [11]:
df['messages'] = df['messages'].astype('int') #Для наглядности меняем тип данных 
print(df.head(5))

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90        83  19915.42         0
1   85.0   516.75        56  22696.96         0
2   77.0   467.66        86  21060.45         0
3  106.0   745.53        81   8437.39         1
4   66.0   418.74         1  14502.75         0


In [12]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


## Разбейте данные на выборки

In [13]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)

#Разбиваем на обучающую и валидо-тестовую V2

In [14]:
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)
#Разбиваем на валидоционную и тестовую выборки пополам v2

In [15]:
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']
#выделяем признаки и цель V2

In [16]:
df.shape

(3214, 5)

In [17]:
df_train.shape #60% от изначального фрейма


(1928, 5)

In [18]:
df_valid_test.shape  #40% от изначального фрейма и делим его пополам далее

(1286, 5)

In [19]:
df_valid.shape # OK!

(643, 5)

## Исследуйте модели

Модель RandomForestRegressor

In [20]:
%%time

best_accuracy = 0 
best_depth = 0 

for depth in range (1, 11):
    for est in range(5, 51, 5):
        for sample in range(2,5):
            model = RandomForestClassifier(random_state=12345,n_estimators=est, max_depth=depth, min_samples_leaf=sample) 
            model.fit(train_features, train_target)
            accuracy = model.score(valid_features, valid_target)
            if accuracy > best_accuracy: 
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
                           

print('Количество деревьев:', best_est, 'Максимальная глубина:', best_depth, 'мин. объектов в узле:', best_sample )
print('Качество', best_accuracy)

Количество деревьев: 35 Максимальная глубина: 6 мин. объектов в узле: 2
Качество 0.8087091757387247
CPU times: total: 17.1 s
Wall time: 19.2 s


<b> Модель DecisionTreeClassifier </b>

In [21]:
%%time
best_accuracy = 0 
best_depth = 0 

for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(train_features, train_target)
    accuracy= model.score(valid_features, valid_target)
    if accuracy > best_accuracy: 
        best_accuracy = accuracy
        best_depth = depth       
               
    
print('Глубина дерева:', best_depth)
print('Качество', best_accuracy)

Глубина дерева: 3
Качество 0.7853810264385692
CPU times: total: 62.5 ms
Wall time: 74 ms


<b> Модель LogisticRegression</b>

In [22]:
%%time
model = LogisticRegression(solver='lbfgs')    
model.fit(train_features, train_target)  
accuracy = model.score(valid_features, valid_target)

print('Качество', accuracy)

Качество 0.7107309486780715
CPU times: total: 15.6 ms
Wall time: 30.9 ms


## Проверьте модель на тестовой выборке

In [23]:
model = RandomForestClassifier(n_estimators=35, max_depth=6, min_samples_leaf=2, random_state=12345 )
model.fit(train_features, train_target)
accuracy = model.score(test_features, test_target)
print('Качество:', accuracy)

Качество: 0.7962674961119751


In [24]:
model = RandomForestClassifier(n_estimators=35, max_depth=6, min_samples_leaf=2, random_state=12345 )
train_valid_df = pd.concat([df_train, df_valid], ignore_index=True)
train_valid_features = train_valid_df.drop(['is_ultra'], axis=1)
train_valid_target = train_valid_df['is_ultra']
model.fit(train_valid_features, train_valid_target)
accuracy = model.score(test_features, test_target)
print('Качество:', accuracy)

Качество: 0.7916018662519441


Вывод о работе с данными: В ходе анализа и работы с данными были применены три модели обучения с вычисляющмися в цикле гиперпараметрами моделей. RandomForest показал наилучшую эффективность accuracy 0.796 на тестовой выборке. LogisticRegression, DecisionTreeClassifier показали результаты хуже - 0.68 и 0.76 соответственно на тестовых выборках. Чтобы улучшать модели возможно потребуются более расширенные значения гиперпараметров и добавление новых признаков по клиентам.